In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility


from scipy.misc import toimage
from scipy.misc import imresize
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, ZeroPadding2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [ ]:
img_rows = 32
img_cols = 100
nb_classes = 12304
nb_epoch = 1

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(str(X_train.shape))

toimage(X_train[0]).save("output2.jpg")

X_train = np.array(map(lambda x: imresize(x, (32, 100)), X_train))
X_test = np.array(map(lambda x: imresize(x, (32, 100)), X_test))

print(str(X_train.shape))
print(str(X_train[0].shape))

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)

print(X_train[0].shape)

toimage(X_train[0].reshape(32, 100)).save('output.jpg')

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')



# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()


model.add(ZeroPadding2D((2,2), input_shape=(1, img_rows, img_cols)))
model.add(Convolution2D(64, 5, 5, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((2,2)))
model.add(Convolution2D(128, 5, 5, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation = 'relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.50))

model.add(Dense(nb_classes, activation = 'softmax'))

print("Compiling model")

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print("Fitting model")

model.fit(X_train, Y_train, batch_size=100, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)

model.save_weights("model_output.hdf5", overwrite = True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

(60000, 28, 28)
(60000, 32, 100)
(32, 100)
X_train shape: (60000, 1, 32, 100)
(1, 32, 100)
60000 train samples
10000 test samples
Compiling model
Fitting model
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 3390/60000 [>.............................] - ETA: 431s - loss: 2.4642 - acc: 0.2985